In [10]:
from __future__ import print_function
from sklearn import datasets
from terminaltables import AsciiTable
import sys
import os
import math
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Import helper functions


In [11]:
class NeuralNet():
    """Generic class to model NeuralNet
    """
    def __init__(self,optimizer,loss):
        # initialize Neural Net superclass
        self.loss      = loss
        self.optimizer = optimizer
        self.layers    = []
        
    def addLayer(self,layer):
        # if there are layers int he network already then, 
        #     set new layers input shape as previous layers output shape
        if self.layers:
            layer.input_shape = self.layers[-1].output_shape
            
        # set optimizer
        layer.initialize(self.optimizer)
    
    def fit(self, X, y, n_epochs, batch_size):

        # Convert to one-hot encoding
        y = to_categorical(y.astype("int"))

        n_samples = np.shape(X)[0]
        n_batches = int(n_samples / batch_size)
        
        bar = progressbar.ProgressBar(widgets=bar_widgets)
        for _ in bar(range(n_epochs)):
 #           idx = range(n_samples)
            idx = np.arange(n_samples)
            np.random.shuffle(idx)

            batch_t_error = 0   # Mean batch training error
            for i in range(n_batches):
                X_batch = X[idx[i*batch_size:(i+1)*batch_size]]
                y_batch = y[idx[i*batch_size:(i+1)*batch_size]]
                loss, _ = self.train_on_batch(X_batch, y_batch)
                batch_t_error += loss

            # Save the epoch mean error
            self.errors["training"].append(batch_t_error / n_batches)
            if self.X_val.any():
                # Calculate the validation error
                y_val_p = self._forward_pass(self.X_val)
                validation_loss = np.mean(self.loss_function.loss(self.y_val, y_val_p))
                self.errors["validation"].append(validation_loss)

        return self.errors["training"], self.errors["validation"]
    
    def _forward_pass(self, X, training=True):
        # Calculate the output of the NN. The output of layer l1 becomes the
        # input of the following layer l2
        layer_output = X
        for layer in self.layers:
            layer_output = layer.forward_pass(layer_output, training)

        return layer_output

    def _backward_pass(self, loss_grad):
        # Propogate the gradient 'backwards' and update the weights
        # in each layer
        acc_grad = loss_grad
        for layer in reversed(self.layers):
            acc_grad = layer.backward_pass(acc_grad)
            
    # Use the trained model to predict labels of X
    def predict(self, X):
        return self._forward_pass(X, training=False)